<a href="https://colab.research.google.com/github/Juanfelipepast2/MetNumUN2024I/blob/main/Lab7/jpastranar_Lab7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple iteration for systems of linear equations

First, generate a random diagonally dominant matrix, for testing.

In [1]:
import numpy as np
rndm = np.random.RandomState(1234)

n = 10
A = rndm.uniform(size=(n, n)) + np.diagflat([15]*n)
b = rndm.uniform(size=n)

# I.  Jacobi iteration

Given

$$
A x = b
$$

separate the diagonal part $D$,

$$ A = D + (A - D) $$

and write

$$
x = D^{-1} (D - A) x + D^{-1} b\;.
$$

Then iterate

$$
x_{n + 1} = B x_{n} + c\;,
$$

where

$$
B = D^{-1} (A - D) \qquad \text{and} \qquad c = D^{-1} b
$$


Let's construct the matrix and the r.h.s. for the Jacobi iteration

In [2]:
diag_1d = np.diag(A)

B = -A.copy()
np.fill_diagonal(B, 0)

D = np.diag(diag_1d)
invD = np.diag(1./diag_1d)
BB = invD @ B
c = invD @ b

In [3]:
# sanity checks
from numpy.testing import assert_allclose

assert_allclose(-B + D, A)


# xx is a "ground truth" solution, compute it using a direct method
xx = np.linalg.solve(A, b)

np.testing.assert_allclose(A@xx, b)
np.testing.assert_allclose(D@xx, B@xx + b)
np.testing.assert_allclose(xx, BB@xx + c)

Check that $\| B\| \leqslant 1$:

In [4]:
np.linalg.norm(BB)

0.36436161983015336

### Do the Jacobi iteration

In [5]:
n_iter = 50

x0 = np.ones(n)
x = x0
for _ in range(n_iter):
    x = BB @ x + c

In [7]:
# Check the result:

A @ x - b

array([ 1.11022302e-16,  0.00000000e+00, -2.22044605e-16, -1.11022302e-16,
        1.11022302e-16,  0.00000000e+00, -2.42861287e-17,  0.00000000e+00,
       -2.77555756e-17,  1.11022302e-16])

### Task I.1

Collect the proof-of-concept above into a single function implementing the Jacobi iteration. This function should receive the r.h.s. matrix $A$, the l.h.s. vector `b`, and the number of iterations to perform.


The matrix $A$ in the illustration above is strongly diagonally dominant, by construction.
What happens if the diagonal matrix elements of $A$ are made smaller? Check the convergence of the Jacobi iteration, and check the value of the norm of $B$.

(20% of the total grade)


In [8]:
# ... ENTER YOUR CODE HERE ...
import numpy as np

def jacobi_iteration(A, b, num_iterations):
    n = len(b)
    x = np.zeros(n)

    D = np.diag(np.diag(A))
    R = A - D

    for _ in range(num_iterations):
        x_new = (b - np.dot(R, x)) / np.diag(D)
        x = x_new

    return x

def check_convergence(A):
    D = np.diag(np.diag(A))
    R = A - D
    B = np.linalg.inv(D).dot(R)
    norm_B = np.linalg.norm(B, ord=np.inf)

    return norm_B

# Example usage
A = np.array([[10, 2, 3],
              [4, 15, 6],
              [7, 8, 20]])
b = np.array([1, 2, 3])
num_iterations = 10

solution = jacobi_iteration(A, b, num_iterations)
norm_B = check_convergence(A)

print("Solution using Jacobi iteration:", solution)
print("Norm of matrix B:", norm_B)

Solution using Jacobi iteration: [0.05360242 0.07813839 0.09829767]
Norm of matrix B: 0.75


# II. Seidel's iteration.

##### Task II.1

Implement the Seidel's iteration.

Test it on a random matrix. Study the convergence of iterations, relate to the norm of the iteration matrix.

(30% of the total grade)

In [9]:
# ... ENTER YOUR CODE HERE ...
import numpy as np

def gauss_seidel_iteration(A, b, num_iterations):
    n = len(b)
    x = np.zeros(n)

    for _ in range(num_iterations):
        x_new = np.copy(x)
        for i in range(n):
            x_new[i] = (b[i] - np.dot(A[i, :i], x_new[:i]) - np.dot(A[i, i + 1:], x[i + 1:])) / A[i, i]
        x = x_new

    return x

def study_convergence(A):
    L = np.tril(A, k=-1)
    U = np.triu(A, k=1)
    D = np.diag(np.diag(A))

    T = -np.linalg.inv(L + D).dot(U)
    norm_T = np.linalg.norm(T, ord=np.inf)

    return norm_T

# Example usage
np.random.seed(42)
random_matrix = np.random.rand(3, 3)
random_vector = np.random.rand(3)
num_iterations = 10

solution_seidel = gauss_seidel_iteration(random_matrix, random_vector, num_iterations)
norm_T = study_convergence(random_matrix)

print("Solution using Gauss-Seidel iteration:", solution_seidel)
print("Norm of matrix T:", norm_T)

Solution using Gauss-Seidel iteration: [ 0.2000545   5.17716694 -5.86584675]
Norm of matrix T: 22.965739532413963


# III. Minimum residual scheme

### Task III.1

Implement the $\textit{minimum residual}$ scheme: an explicit non-stationary method, where at each step you select the iteration parameter $\tau_n$ to minimize the residual $\mathbf{r}_{n+1}$ given $\mathbf{r}_n$. Test it on a random matrix, study the convergence to the solution, in terms of the norm of the residual and the deviation from the ground truth solution (which you can obtain using a direct method). Study how the iteration parameter $\tau_n$ changes as iterations progress.

(50% of the grade)

In [10]:
# ... ENTER YOUR CODE HERE ...
import numpy as np

def minimum_residual_scheme(A, b, num_iterations):
    n = len(b)
    x = np.zeros(n)
    x_truth = np.linalg.solve(A, b)

    for _ in range(num_iterations):
        r = b - A.dot(x)
        tau = np.dot(r, A.dot(r)) / np.dot(A.dot(r), A.dot(r))
        x = x + tau * r

        residual_norm = np.linalg.norm(b - A.dot(x))
        deviation_truth = np.linalg.norm(x - x_truth)

        print(f"Iteration: {_+1} - Residual Norm: {residual_norm}, Deviation from Truth: {deviation_truth}, Tau: {tau}")

    return x

# Example usage
np.random.seed(42)
random_matrix = np.random.rand(3, 3)
random_vector = np.random.rand(3)
num_iterations = 5

solution_minimum_residual = minimum_residual_scheme(random_matrix, random_vector, num_iterations)


Iteration: 1 - Residual Norm: 0.6402891257761572, Deviation from Truth: 8.379739397787493, Tau: 0.7711771263434366
Iteration: 2 - Residual Norm: 0.6066785342518726, Deviation from Truth: 7.304991999196145, Tau: -1.7000294817348063
Iteration: 3 - Residual Norm: 0.5944799340897743, Deviation from Truth: 7.473538055144704, Tau: 0.31411795990756675
Iteration: 4 - Residual Norm: 0.592822897061918, Deviation from Truth: 7.358003698906767, Tau: -0.20231136582600817
Iteration: 5 - Residual Norm: 0.5924147256123201, Deviation from Truth: 7.401822719639131, Tau: 0.0788902878079138
